In [1]:
import json
import os
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

from datetime import date
from dateutil import parser
from dateutil.relativedelta import relativedelta

In [2]:
def addyears(date_str, num):
    d = parser.parse(date_str).date()
    return str(d + relativedelta(years=num))
def adddays(date_str, num):
    d = parser.parse(date_str).date()
    return str(d + relativedelta(days=num))
def save_pic(driver, ticker, start_date, end_date, file_name):
    driver.find_element_by_id('navsearchtext').send_keys(Keys.CONTROL + 'a' + Keys.NULL, ticker)
    driver.find_element_by_id('navsearchbutton').click()
    select = Select(driver.find_element_by_id('dataRange'))
    select.select_by_visible_text('Select Start/End')
    driver.find_element_by_id('start').send_keys(Keys.CONTROL + 'a' + Keys.NULL, start_date)
    driver.find_element_by_id('submitButton').click() # otherwise the end date wont change
    driver.find_element_by_id('end').send_keys(Keys.CONTROL + 'a' + Keys.NULL, end_date)
    driver.find_element_by_id('submitButton').click()
    driver.get_screenshot_as_file(file_name)
    
def save_charts(trade, driver):
    ticker = trade.EnterTicker
    file_period_name = trade.EnterDate.replace('-','_')
    enter_period = trade.EnterDate
    enter_one_year_back = addyears(enter_period, -1)
    enter_file_name = str.format('{0}_{1}_Entry.png', file_period_name, ticker.replace('/', '-'))
    exit_period = trade.ExitDate
    exit_one_year_back = addyears(exit_period, -1)
    exit_file_name = str.format('{0}_{1}_Exit.png', file_period_name, ticker.replace('/', '-'))
    p10_period = adddays(exit_period, 15) # two weeks
    p10_one_year_back = addyears(p10_period, -1)
    p10_file_name = str.format('{0}_{1}_Exit_p10.png', file_period_name, ticker.replace('/', '-'))
    p30_period = adddays(exit_period, 90) # three months
    p30_one_year_back = addyears(p30_period, -1)
    p30_file_name = str.format('{0}_{1}_Exit_p30.png', file_period_name, ticker.replace('/', '-'))
    save_pic(driver, ticker, enter_one_year_back, enter_period, enter_file_name)
    save_pic(driver, ticker, exit_one_year_back, exit_period, exit_file_name)
    save_pic(driver, ticker, p10_one_year_back, p10_period, p10_file_name)
    save_pic(driver, ticker, p30_one_year_back, p30_period, p30_file_name)

In [4]:
driver = webdriver.Firefox()
driver.implicitly_wait(10) # seconds
driver.get("http://stockcharts.com")
account_json = 'C:\\Tools\\stockchart.json'
with open(account_json) as data_file:    
    account_info = json.load(data_file)
driver.find_element_by_link_text('Login').click()
driver.find_element_by_id('form_UserID').send_keys(account_info['user_name'])
driver.find_element_by_name('form_UserPassword').send_keys(account_info['password'])
driver.find_element_by_id('loginbutton').submit()

In [5]:
trade_file = "C:\\temp\\trade_result.csv"
trade_df = pd.read_csv(trade_file, parse_dates=True)
for index in range(0, trade_df.EnterDate.size):
    trade = trade_df.iloc[index]
    save_charts(trade, driver)
    
